In [1]:
import pyhf
import numpy as np
import matplotlib.pyplot as plt
from pyhf.contrib.viz import brazil

import ROOT
from ROOT import TGraph,TCanvas,TLegend

Welcome to JupyROOT 6.20/06


In [2]:
def plotUpperLimits(values,limits,XaxisTitle,YaxisTitle,outfigure,theoryprediction = None,logy=False,logx=False):
    # see CMS plot guidelines: https://ghm.web.cern.ch/ghm/plots/
 
    N = len(values)
    yellow = TGraph(2*N)    # yellow band
    green = TGraph(2*N)     # green band
    median = TGraph(N)      # median line
    if theoryprediction:
        theory = TGraph(N)      # theory line
 
    up2s = [ ]
    dn2s = [ ]
    for i in range(N):
        limit = limits[i]
        up2s.append(limit[4])
        dn2s.append(limit[0])
        yellow.SetPoint(    i,    values[i], limit[4] ) # + 2 sigma
        green.SetPoint(     i,    values[i], limit[3] ) # + 1 sigma
        median.SetPoint(    i,    values[i], limit[2] ) # median
        if theoryprediction:
            theory.SetPoint(    i,    values[i], theoryprediction[i] ) # median
        green.SetPoint(  2*N-1-i, values[i], limit[1] ) # - 1 sigma
        yellow.SetPoint( 2*N-1-i, values[i], limit[0] ) # - 2 sigma
 
    W = 800
    H  = 800
    T = 0.08*H
    B = 0.12*H
    L = 0.12*W
    R = 0.04*W
    c = TCanvas("c","c",100,100,W,H)
    c.SetFillColor(0)
    c.SetBorderMode(0)
    c.SetFrameFillStyle(0)
    c.SetFrameBorderMode(0)
    c.SetLeftMargin( L/W )
    c.SetRightMargin( R/W )
    c.SetTopMargin( T/H )
    c.SetBottomMargin( B/H )
    c.SetTickx(0)
    c.SetTicky(0)
    c.SetGrid()
    if logy:
        c.SetLogy()
    if logx:
        c.SetLogx()
    c.cd()
    
    frame = c.DrawFrame(min(values)*0.8,min(dn2s)*0.1, max(values)*2, max(up2s)*10)
    frame.GetYaxis().CenterTitle()
    frame.GetYaxis().SetTitleSize(0.05)
    frame.GetXaxis().SetTitleSize(0.05)
    frame.GetXaxis().SetLabelSize(0.03)
    frame.GetYaxis().SetLabelSize(0.03)
    frame.GetYaxis().SetTitleOffset(1.1)
    frame.GetXaxis().SetNdivisions(508)
    frame.GetYaxis().CenterTitle(True)
    frame.GetYaxis().SetTitle(YaxisTitle)
#    frame.GetYaxis().SetTitle("95% upper limit on #sigma #times BR / (#sigma #times BR)_{SM}")
    frame.GetXaxis().SetTitle(XaxisTitle)
    if logy:
        frame.SetMinimum(min(dn2s)*0.1)
        frame.SetMaximum(max(up2s)*10)
    else:
        frame.SetMinimum(0)
        frame.SetMaximum(max(up2s)*1.2)
    frame.GetXaxis().SetLimits(min(values),max(values))
 
    yellow.SetFillColor(ROOT.kOrange)
    yellow.SetLineColor(ROOT.kOrange)
    yellow.SetFillStyle(1001)
    yellow.Draw('F')
 
    green.SetFillColor(ROOT.kGreen+1)
    green.SetLineColor(ROOT.kGreen+1)
    green.SetFillStyle(1001)
    green.Draw('Fsame')
 
    median.SetLineColor(1)
    median.SetLineWidth(2)
    median.SetLineStyle(2)
    median.Draw('Lsame')

    if theoryprediction:
        theory.SetLineColor(2)
        theory.SetLineWidth(2)
        theory.SetLineStyle(1)
        theory.Draw('Lsame')
 
#     CMS_lumi.CMS_lumi(c,14,11)
    ROOT.gPad.SetTicks(1,1)
    frame.Draw('sameaxis')
 
    x1 = 0.1
    x2 = 0.3
    y2 = 0.9
    y1 = 0.7
    legend = TLegend(0.2,0.7,0.5,0.9)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.03)
    legend.SetTextFont(42)
    legend.AddEntry(median, "Asymptotic CL_{s} expected",'L')
    legend.AddEntry(green, "#pm 1 std. deviation",'f')
#    legend.AddEntry(green, "Asymptotic CL_{s} #pm 1 std. deviation",'f')
    legend.AddEntry(yellow,"#pm 2 std. deviation",'f')
    if theoryprediction:
        legend.AddEntry(theory, "theory prediciotn",'L')
#    legend.AddEntry(green, "Asymptotic CL_{s} #pm 2 std. deviation",'f')
    legend.Draw()
    
    c.Update()
    c.Print(outfigure)
 
    Graphs = [yellow,median,green,]
    if theoryprediction:
        Graphs = [yellow,median,green,theory,legend]

    return c,Graphs


In [3]:
limits_const_air=[]
limits_const_vac=[]
limits_maxwell_air=[]
limits_maxwell_vac=[]
MDM=[0.005,0.05,0.1,0.2,0.5,1,10,100]
DMmass=['0p005','0p05','0p1','0p2','0p5','1','10','100']

#const_air
for iDMmass in DMmass:
    flimits = ROOT.TFile.Open("UL_const_air/higgsCombine" + "const_" + iDMmass + "_air" + ".AsymptoticLimits.mH120.root")
    tlimits = flimits.Get("limit")
    limitsnow=[]
    
    nEvents = tlimits.GetEntries()
    for i in range(nEvents):
        tlimits.GetEntry(i)
        limitsnow.append(tlimits.limit)
    
    limits_const_air.append(limitsnow)
    flimits.Close()
    
#const_vac
for iDMmass in DMmass:
    flimits = ROOT.TFile.Open("UL_const_vac/higgsCombine" + "const_" + iDMmass + "_vac" + ".AsymptoticLimits.mH120.root")
    tlimits = flimits.Get("limit")
    limitsnow=[]
    
    nEvents = tlimits.GetEntries()
    for i in range(nEvents):
        tlimits.GetEntry(i)
        limitsnow.append(tlimits.limit)
    
    limits_const_vac.append(limitsnow)
    flimits.Close()
    
#maxwell_air
for iDMmass in DMmass:
    flimits = ROOT.TFile.Open("UL_maxwell_air/higgsCombine" + "maxwell_" + iDMmass + "_air" + ".AsymptoticLimits.mH120.root")
    tlimits = flimits.Get("limit")
    limitsnow=[]
    
    nEvents = tlimits.GetEntries()
    for i in range(nEvents):
        tlimits.GetEntry(i)
        limitsnow.append(tlimits.limit)
    
    limits_maxwell_air.append(limitsnow)
    flimits.Close()

#maxwell_vac
for iDMmass in DMmass:
    flimits = ROOT.TFile.Open("UL_maxwell_vac/higgsCombine" + "maxwell_" + iDMmass + "_vac" + ".AsymptoticLimits.mH120.root")
    tlimits = flimits.Get("limit")
    limitsnow=[]
    
    nEvents = tlimits.GetEntries()
    for i in range(nEvents):
        tlimits.GetEntry(i)
        limitsnow.append(tlimits.limit)
    
    limits_maxwell_vac.append(limitsnow)
    flimits.Close()

nsigHist_const=[]
nsigHist_maxwell=[]
fHist = ROOT.TFile.Open("./Hist.root")
for iDMmass in DMmass:
    Hist = fHist.Get("h_const_"+iDMmass)
    nsigHist_const.append(Hist.Integral())
for iDMmass in DMmass:
    Hist = fHist.Get("h_maxwell_"+iDMmass)
    nsigHist_maxwell.append(Hist.Integral())

nbkgHist_air=fHist.Get("h_air").Integral()
nbkgHist_vac=fHist.Get("h_vac").Integral()

In [4]:
#Dark Matter
rhoDM=0.3#GeV/cm^3
Vbox=1* 100**3#1m^3

NDM=[]
for m in MDM:
    NDM.append(rhoDM*Vbox/m)
    
#signal
#Nsig/eff=NDM*I*sigma*T
eff_const=[]
for i in range(len(nsigHist_const)):
    eff=nsigHist_const[i]/10000000.
    if i == 0:
        eff *= 10
    else:
        eff *= 1000
    eff_const.append(eff)
    
eff_maxwell=[]
for i in range(len(nsigHist_maxwell)):
    eff=nsigHist_maxwell[i]/10000000.
    if i == 0:
        eff *= 10
    else:
        eff *= 1000
    eff_maxwell.append(eff)

I=1/60. # s^-1 cm^-2
T=3* 10**7 #s = 1year

sigmalimits_const_air=[]
sigmalimits_const_vac=[]
sigmalimits_maxwell_air=[]
sigmalimits_maxwell_vac=[]

#const_air
for iDM in range(len(DMmass)):
    limitsnow=[]
    for iCL in range(5):
        limitsnow.append(limits_const_air[iDM][iCL]*nsigHist_const[iDM]/eff_const[iDM]/I/T/NDM[iDM])
    sigmalimits_const_air.append(limitsnow)
    
#const_vac
for iDM in range(len(DMmass)):
    limitsnow=[]
    for iCL in range(5):
        limitsnow.append(limits_const_vac[iDM][iCL]*nsigHist_const[iDM]/eff_const[iDM]/I/T/NDM[iDM])
    sigmalimits_const_vac.append(limitsnow)
    
#maxwell_air
for iDM in range(len(DMmass)):
    limitsnow=[]
    for iCL in range(5):
        limitsnow.append(limits_maxwell_air[iDM][iCL]*nsigHist_maxwell[iDM]/eff_maxwell[iDM]/I/T/NDM[iDM])
    sigmalimits_maxwell_air.append(limitsnow)
    
#maxwell_vac
for iDM in range(len(DMmass)):
    limitsnow=[]
    for iCL in range(5):
        limitsnow.append(limits_maxwell_vac[iDM][iCL]*nsigHist_maxwell[iDM]/eff_maxwell[iDM]/I/T/NDM[iDM])
    sigmalimits_maxwell_vac.append(limitsnow)

In [5]:
can,_ = plotUpperLimits(MDM,sigmalimits_const_air,"DM mass (GeV)","95% upper limit on #sigma_{#mu,DM} (cm^{2})","figure/UL_const_air.eps",logy=True,logx=True)
can,_ = plotUpperLimits(MDM,sigmalimits_const_vac,"DM mass (GeV)","95% upper limit on #sigma_{#mu,DM} (cm^{2})","figure/UL_const_vac.eps",logy=True,logx=True)
can,_ = plotUpperLimits(MDM,sigmalimits_maxwell_air,"DM mass (GeV)","95% upper limit on #sigma_{#mu,DM} (cm^{2})","figure/UL_maxwell_air.eps",logy=True,logx=True)
can,_ = plotUpperLimits(MDM,sigmalimits_maxwell_vac,"DM mass (GeV)","95% upper limit on #sigma_{#mu,DM} (cm^{2})","figure/UL_maxwell_vac.eps",logy=True,logx=True)

Info in <TCanvas::Print>: eps file figure/UL_const_air.eps has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: eps file figure/UL_const_vac.eps has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: eps file figure/UL_maxwell_air.eps has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: eps file figure/UL_maxwell_vac.eps has been created


In [6]:
nbkgHist_air

1146508770.0

In [7]:
nbkgHist_vac

1144378120.0

In [8]:
eff_const

[0.27098228749909253,
 0.29560830445388564,
 0.27659010347613366,
 0.2501312021017075,
 0.21473410029411316,
 0.18675220268964765,
 0.11104290096759796,
 0.08438850038051605]

In [9]:
err_eff_const=[]
for i in eff_const:
    err_eff_const.append(np.sqrt(i*(1-i)/10000000))
err_eff_const

[0.00014055279696998266,
 0.0001442997002047421,
 0.00014145247192438716,
 0.00013695458511377495,
 0.00012985506014976503,
 0.00012323790710662544,
 9.935410163264433e-05,
 8.790169587900093e-05]

In [10]:
eff_maxwell

[0.2710614874987155,
 0.2954848041568999,
 0.27644510129024275,
 0.2499060976564884,
 0.21460949890613557,
 0.18655619782209396,
 0.11098340034484863,
 0.08424110023975372]